In [74]:
import spacy
from spacy import displacy
import pandas as pd
#nlp = spacy.load('en_core_web_md')
#nlp = spacy.load('en_core_web_sm')
#nlp = spacy.load("en_scispacy_core_web_sm")

# Scispacy post-release
nlp = spacy.load('en_core_sci_md')
#ner = spacy.load('en_ner_craft_md')
ner = spacy.load('en_ner_jnlpba_md')
nlp.replace_pipe('ner', ner.pipeline[0][1])

# The individual entity type names (e.g. CELL_TYPE, PROTEIN, etc. need to be 
# added to the core nlp vocab in order to avoid "label not in StringStore errors")
ner_types = sorted(list(set([typ.split('-')[-1] for typ in ner.pipeline[0][1].move_names])))
for typ in ner_types:
    nlp.vocab.strings.add(typ)
ner_types

['CELL_LINE', 'CELL_TYPE', 'DNA', 'O', 'PROTEIN', 'RNA']

In [120]:
# text = u"Autonomous cars shift insurance liability toward manufacturers")
# text = u'Apple is looking at buying U.K. startup for $1 billion')
#text = u"We identify a core transcriptional profile within the CD69+ subset of memory CD4+ and CD8+ T cells in lung and spleen that is distinct from that of CD69−TEM cells in tissues and circulation, and defines human TRM based on homology to the transcriptional profile of mouse CD8+TRM."
#text = u"Aging is associated with a decrease in naïve (T(N)) and central memory (T(CM)), and an accumulation of effector memory (T(EM) and T(EMRA)) T cell subsets. Previously, we have demonstrated an increased sensitivity of T(N) and T(CM) CD4+ and CD8+ T cells in aging to TNF-alpha-induced apoptosis. In this investigation, we examined whether similar differential sensitivity is applicable to CD95-mediated apoptosis. We show that T(N) and T(CM) CD4+ and CD8+ T cells from aged subjects are significantly more sensitive to CD95-mediated apoptosis. Increased apoptosis is associated with increased activation of caspase-8 and caspase-3. Both caspase-8 and caspase-3 inhibitors blocked CD95-mediated apoptosis and activation of caspase-8 and caspase-3 in T(N) and T(CM) CD4+ and CD8+ T cells. No significant difference was observed in apoptosis or in activation of caspase-8 and caspase-3 in T(EM) and T(EMRA) CD4+ and CD8+ T cells"
#text = u"T cell immunity is often defined in terms of memory lymphocytes that use the blood to access a range of organs. T cells are involved in two patterns of recirculation. In one, the cells shuttle back and forth between blood and secondary lymphoid organs, whereas in the second, memory cells recirculate between blood and nonlymphoid tissues. The latter is a means by which blood T cells control peripheral infection. It is now clear that there exists a distinct memory T cell subset that is absent from blood but found within nonlymphoid tissues. These nonrecirculating tissue-resident memory T (TRM) cells develop within peripheral compartments and never spread beyond their point of lodgement"
#text = u"The immune responses of naive and different memory subsets of CD4(+) and CD8(+) T cells to human herpesvirus 6 (HHV-6) have not been previously investigated. We show that HHV-6A induces cell division, as measured by 5,6-carboxyfluorescein succinimidyl ester dye and flow cytometry, predominantly in two populations of effector memory CD4(+) and CD8(+) T cells ( T (EM) and T (EMRA) )"
#text = u"Tissue-resident (TRM) T cells are like effector (EM) T lymphocytes; because a T cell (TN) is a T-lymphocyte."
#text = u"In this study, we evaluated redox status in memory CD4 + T cells and plasma of patients with psoriasis and its correlation with IL-17 response. To this end, memory T cells were isolated from 10 patients with psoriasis and 10 controls."
#text = u"Aging is associated with a decrease in naïve (T (N)) and central memory (T (CM)), and an accumulation of effector memory (T (EM) and T (EMRA)) T cell subsets"
#text = u"Aging is associated with a decrease in naïve (TN) and central memory (TCM), and an accumulation of effector memory (T (EM) and T (EMRA)) T cell subsets"
#text = u"Sodium chloride (NaCl) has been proposed as a driving factor in autoimmune diseases through the induction of pathogenic CD4<sup>+</sup> T helper cells that produce interleukin-17 (Th17 cells). This study investigated the effects of NaCl on inflammatory arthritis in mice and humans"
#text = u"Foxp3<sup>+</sup>CD4<sup>+</sup> regulatory T (T<sub>reg</sub>) cells are essential for preventing fatal autoimmunity and safeguard immune homeostasis in vivo. While expression of the transcription factor Foxp3 and IL-2 signals are both required for the development and function of T<sub>reg</sub> cells"
#text = u"Mucosa-associated invariant (MAIT) T cells are a subset of innate-like T lymphocytes known for their ability to respond to MHC-related protein 1"
#text = u"MAIT cells are expanded in salivary glands of patients with Sjogren's syndrome and are IL-17 polarized. IL-7 and IL-23 induce IL-17 production activating two different pathways: IL-7 stimulation induces in fact a significant STAT3 and HIF1alpha upregulation, conversely, IL-23 stimulation significantly induces RORc overexpression in MAIT cells of patients with Sjogren's syndrome"
#text = u"Robust in vitro systems are important in elucidating mechanisms regarding the heterogenetic nature of in vivo immune responses and contribute with knowledge to design good in vivo experiments. In this study, we show that initial Th1 and in particular Th2 polarization was negatively influenced by even small percentages (<5-10%) of 'polluting' memory CD4+ T cells producing IL-5, IL-9, IL-10 IL-13, IL-21, IL-22, IL-31 and IFN-γ that are normally found after standard immunomagnetic bead separation of naïve CD4+ T cell. By using an alternative protocol for immunomagnetic bead separation of naïve CD4+ T cells, we found that cultures of the obtained >99% naïve CD4+ T cells resulted in better Th1 and Th2 polarization with significant reduced fractions of IL-4+ and IFN-γ+ CD4+ T cells, respectively. Moreover, the Th2 primed >99% naïve CD4+ T cells showed significantly higher ratio of IL-4+:IFN-γ+ (>4 fold) and GATA-3:+T-bet+ (>3 fold) CD4+ T cells when compared with the standard purified >90-95% naïve CD4+ T cells primed under the same culture conditions. This suggests immunomagnetic bead separation, a low cost and easy available technique, with few modifications to the manufacturer's protocol as an attractive alternative for laboratories not having a cell sorter. Taken together, we report that it is essential to use rigorously purified (>99%) naïve CD4+ T cells for optimal initial in vitro Th1 and Th2 priming."
#text = u"Four days later, Th1 vs. Th2 cell differentiation was assessed by measuring IFN-γ or IL-4 production, respectively, by restimulated recovered 5CC7 cells.. LPS-treated DC induced a strongly biased Th1 cell response, whereas interaction with papain-treated DC led to a Th2 cell skewed response (Figure 1A–C). We did not detect cells producing IL-17 under these conditions (data not shown) and note that the fraction of cells showing either a Th1 or Th2 cell effector state is in accord with other in vivo immunization findings."
text = u"Th1 cells, which produce interferon (IFN)-gamma, interleukin (IL)-2 and tumor necrosis factor (TNF)-beta, evoke cell-mediated immunity and phagocyte-dependent inflammation. Th2 cells, which produce IL-4, IL-5, IL-6, IL-9, IL-10, and IL-13, evoke strong antibody responses (including those of the IgE class) and eosinophil accumulation, but inhibit several functions of phagocytic cells (phagocyte-independent inflammation). Both environmental and genetic factors act in concert to determine the Th1 or Th2 polarization. Further, Th1-dominated responses are involved in the pathogenesis of organ-specific autoimmune disorders, Crohn's disease, sarcoidosis, acute kidney allograft rejection, and some unexplained recurrent abortions. In contrast, allergen-specific Th2 responses are responsible for atopic disorders in genetically susceptible individuals."
text

"Th1 cells, which produce interferon (IFN)-gamma, interleukin (IL)-2 and tumor necrosis factor (TNF)-beta, evoke cell-mediated immunity and phagocyte-dependent inflammation. Th2 cells, which produce IL-4, IL-5, IL-6, IL-9, IL-10, and IL-13, evoke strong antibody responses (including those of the IgE class) and eosinophil accumulation, but inhibit several functions of phagocytic cells (phagocyte-independent inflammation). Both environmental and genetic factors act in concert to determine the Th1 or Th2 polarization. Further, Th1-dominated responses are involved in the pathogenesis of organ-specific autoimmune disorders, Crohn's disease, sarcoidosis, acute kidney allograft rejection, and some unexplained recurrent abortions. In contrast, allergen-specific Th2 responses are responsible for atopic disorders in genetically susceptible individuals."

In [121]:
import re
def prep(text):
    text = re.sub('\\<(sup|sub)\\>([a-zA-Z+-]*)\\</(sup|sub)>', '\\2 ', text)
    #text = re.sub('[tT]\\-([cC]ell[s]*|[lL]ymphocyte[s]*)', 'T \\1', text)
    return text
prep(text)

"Th1 cells, which produce interferon (IFN)-gamma, interleukin (IL)-2 and tumor necrosis factor (TNF)-beta, evoke cell-mediated immunity and phagocyte-dependent inflammation. Th2 cells, which produce IL-4, IL-5, IL-6, IL-9, IL-10, and IL-13, evoke strong antibody responses (including those of the IgE class) and eosinophil accumulation, but inhibit several functions of phagocytic cells (phagocyte-independent inflammation). Both environmental and genetic factors act in concert to determine the Th1 or Th2 polarization. Further, Th1-dominated responses are involved in the pathogenesis of organ-specific autoimmune disorders, Crohn's disease, sarcoidosis, acute kidney allograft rejection, and some unexplained recurrent abortions. In contrast, allergen-specific Th2 responses are responsible for atopic disorders in genetically susceptible individuals."

In [122]:
#doc = nlp(text)
#doc, ents = nlp(prep(text)), ner(prep(text))
doc = nlp(prep(text))

In [123]:
# for token in doc:
#     print(token.text, token.dep_, token.head.text, token.head.pos_,
#           [child for child in token.children])

In [124]:
for chunk in doc.noun_chunks:
    print('\t'.join([str(v) for v in [chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text]]))
#pd.DataFrame(df, columns=['chunk.text', 'chunk.root.text', 'chunk.root.dep_', 'chunk.root.head.text'])

Th1 cells	cells	nsubj	evoke
interferon (IFN)-gamma	(IFN)-gamma	dobj	produce
interleukin (IL)-2	(IL)-2	conj	cells
tumor necrosis factor (TNF)-beta	(TNF)-beta	conj	(IL)-2
cell-mediated immunity	immunity	dobj	evoke
phagocyte-dependent inflammation	inflammation	conj	immunity
Th2 cells	cells	nsubj	evoke
IL-4	IL-4	dobj	produce
IL-5	IL-5	conj	IL-4
IL-6	IL-6	conj	IL-4
IL-9	IL-9	conj	IL-4
IL-10	IL-10	conj	IL-4
IL-13	IL-13	conj	IL-4
strong antibody responses	responses	dobj	evoke
eosinophil accumulation	accumulation	conj	responses
several functions	functions	dobj	inhibit
(phagocyte-independent inflammation	inflammation	appos	cells
factors	factors	nsubj	act
the Th1 or Th2 polarization	polarization	dobj	determine
Th1-dominated responses	responses	nsubjpass	involved
some unexplained recurrent abortions	abortions	conj	involved
allergen-specific Th2 responses	responses	nsubj	responsible


In [125]:
displacy.render(doc, style='dep', jupyter=True, options=dict(compact=True))

In [126]:
# for ent in doc.ents:
#     print(ent)

In [127]:
ner_types

['CELL_LINE', 'CELL_TYPE', 'DNA', 'O', 'PROTEIN', 'RNA']

In [128]:
colors = {k: 'grey' for k in ner_types}
colors.update({'PROTEIN': 'green', 'CELL_TYPE': 'red'})
options = {'ents': ner_types, 'colors': colors}
displacy.render(doc, style='ent', jupyter=True, options=options)

### Examples

In [9]:
from IPython.core.display import display, HTML
doc = {
    'text': 'But Google is starting from behind.',
    'ents': [{'start': 4, 'end': 10, 'label': 'ORG'}],
    'title': None
}
html = displacy.render(doc, style='ent', manual=True)
display(HTML(html))

In [ ]:

# def find_node(tokens, p):
#     for token in tokens:
#         if p(token):
#             return token
#     else:
#         children = [t for token in tokens for t in list(token.children)]
#         if not children:
#             return None
#         return find_node(children, p)

# #     def is_t_acronym(t):
# #         hch = list(t.head.children)
# #         return t.pos_ in ['NOUN', 'PROPN'] and len(hch) >= 3 and hch[0].text == '(' and hch[2].text == ')'

# def extract_t_acronym(text):
#     text = re.sub('[()]', '', text)
#     text = [w.strip() for w in text.split()]
#     return [t for t in text if t.startswith('T') and len(t) >= 2]

# macr = {}
# for token in doc:
#     if token.pos_ in ['NOUN', 'PROPN'] and token.lemma_ in ['cell', 'lymphocyte']:
#         is_t_token = find_node([token], lambda t: t.text == 'T') is not None
#         if is_t_token and token.sent.text not in macr:
#             macr[token.sent.text] = extract_t_acronym(token.sent.text)
# macr